In [ ]:
import os
import sys

curr_dir = os.getcwd()
src_dir = os.path.join(os.path.dirname(curr_dir), "src")
sql_dir = os.path.join(os.path.dirname(curr_dir), "sql")
sys.path.append(src_dir)
sys.path.append(sql_dir)

In [ ]:
import warnings
from ast import literal_eval

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from db import get_db
from settings import Path
from sqlalchemy import text

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", 100)

In [ ]:
def fetch_data(source: str):
    """
    Fetches data from a specified source and returns it as a pandas DataFrame.

    Parameters:
    source (str): The name of the source file containing the SQL query.

    Returns:
    pandas.DataFrame: The fetched data as a DataFrame.

    """
    db = get_db()

    with open(os.path.join(sql_dir, source), "r") as f:
        query = text(f.read())
    result = db.execute(query)
    data = result.fetchall()
    columns = result.keys()
    df = pd.DataFrame(data, columns=columns)

    db.close()

    return df

In [ ]:
steamspy_data = fetch_data("get_all_steamspy_data.sql")

In [ ]:
appids_to_remove = steamspy_data[~steamspy_data['appid'].isin(steam_data['appid'])]['appid']
steamspy_data = steamspy_data[~steamspy_data['appid'].isin(appids_to_remove)]
steamspy_data